In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.getcwd()

'C:\\Users\\d.dey\\Documents\\Python Scripts'

In [4]:
os.chdir("C:\\EclipseBytes\\CodeBaseDebo\\sql_excel")

In [5]:
df = pd.read_csv("orderline.txt",sep='\t')

In [6]:
df.shape

(286017, 8)

In [7]:
print(df.head(5))
print('\n')

# check  the data for the product 10834 first 20
print(df[df['productid']==10834].head(3))

   orderlineid  orderid  productid             shipdate             billdate  \
0      1010561  1006414      10834  2011-03-07 00:00:00  2011-03-08 00:00:00   
1      1010562  1006541      11052  2011-01-19 00:00:00  2011-01-20 00:00:00   
2      1010563  1006542      11070  2011-01-19 00:00:00  2011-01-20 00:00:00   
3      1010564  1010154      11196  2009-11-19 00:00:00  2009-11-20 00:00:00   
4      1010565  1009110      11048  2009-11-11 00:00:00  2009-11-12 00:00:00   

   unitprice  numunits  totalprice  
0      18.00         1       18.00  
1      10.00         2       20.00  
2      19.12         1       19.12  
3      14.95         1       14.95  
4      10.00         2       20.00  


    orderlineid  orderid  productid             shipdate             billdate  \
0       1010561  1006414      10834  2011-03-07 00:00:00  2011-03-08 00:00:00   
28      1010589  1008588      10834  2011-03-08 00:00:00  2011-03-09 00:00:00   
65      1010626  1010317      10834  2010-12-09 00:0

In [11]:
# Check sorting by sort_values with ascensding
# group by productid
# aggregations on 2 columns : only np sum here , more can be added
df_agg = df.sort_values(by=['billdate'],ascending=[1]).groupby(['productid']).agg({'numunits':[np.sum],'totalprice':np.sum}).reset_index()
print(df_agg.head(5))

  productid totalprice numunits
                   sum      sum
0     10001    4035.00      727
1     10002    3774.99      364
2     10003    2865.97      320
3     10004    2665.80       34
4     10005   26356.47     1773


In [9]:
df_one_prod = df.sort_values(by=['billdate'],ascending=[1]).groupby(['productid']).agg({'numunits':np.cumsum,'totalprice':np.cumsum}).reset_index()
print(df_one_prod.head(5))

   index  totalprice  numunits
0   5389       19.68         1
1   5432       39.36         2
2   5426       59.04         3
3   6466       78.72         4
4   3252       98.40         5


In [10]:
#df['No_cumm'] =df.groupby(['productid'])['numunits'].apply(lambda x: x.cumsum())

# create the cummulative sum and append to the df
df['No_cumm'] =df.sort_values(by=['productid','billdate','numunits'],ascending=[1,1,1]).groupby(['productid'])['numunits'].transform(np.cumsum)
#df.reset_index()


#print(df.head(5))# calculate the sum of transaction and amount 
df['total_trx'] = df.groupby(['productid'])['numunits'].transform(np.sum)
df['total_amt'] = df.groupby(['productid'])['totalprice'].transform(np.sum)

# displaying the select rows along with the select columns for specific count
print(df.loc[(df['productid']==10006),\
             ['billdate','productid','numunits','No_cumm','total_trx','total_amt']].\
             sort_values(by=['billdate','numunits'],ascending=[1,1])[120:130])


                  billdate  productid  numunits  No_cumm  total_trx  total_amt
44402  2010-11-25 00:00:00      10006         1      121       2329    15414.4
44500  2010-11-25 00:00:00      10006         1      122       2329    15414.4
44753  2010-11-25 00:00:00      10006         1      123       2329    15414.4
44817  2010-11-25 00:00:00      10006         1      124       2329    15414.4
44984  2010-11-25 00:00:00      10006         1      125       2329    15414.4
45092  2010-11-25 00:00:00      10006         1      126       2329    15414.4
45392  2010-11-25 00:00:00      10006         1      127       2329    15414.4
45424  2010-11-25 00:00:00      10006         1      128       2329    15414.4
45521  2010-11-25 00:00:00      10006         1      129       2329    15414.4
45668  2010-11-25 00:00:00      10006         1      130       2329    15414.4


In [47]:
# this method takes long time to compute. always choose the alternate one down below:
def ranker(df):
    df['rank'] = np.arange(len(df)) + 1
    return df

df=df.groupby(['productid','billdate','totalprice']).apply(ranker)

In [ ]:
# ad a 
df['rankProd']= df.sort_values(by=['productid','billdate'],ascending=[1,1]).groupby(['productid']).rank(ascending=1,method='first')

In [12]:
df['No_cumm'] =df.sort_values(by=['rankProd'],ascending=[1]).groupby(['productid'])['numunits'].transform(np.cumsum)

KeyError: 'rankProd'

In [143]:
df.groupby(['productid'])['numunits'].transform(np.cumsum).head(5)

0    1
1    2
2    1
3    1
4    2
Name: numunits, dtype: int64

In [163]:
df[df['productid']==10001].head(6)

,orderlineid,orderid,productid,shipdate,billdate,unitprice,numunits,totalprice,No_cumm,total_trx,total_5trx
48006,1048007,1042778,10001,2012-08-29 00:00:00,2012-08-30 00:00:00,13.35,1,13.35,1,727,5
48069,1048070,1042792,10001,2012-08-29 00:00:00,2012-08-30 00:00:00,13.50,1,13.50,2,727,5
48808,1048809,1042112,10001,2012-09-05 00:00:00,2012-09-06 00:00:00,13.35,1,13.35,3,727,5
49565,1049566,1042297,10001,2012-09-10 00:00:00,2012-09-11 00:00:00,13.50,1,13.50,4,727,5
49567,1049568,1043014,10001,2012-09-12 00:00:00,2012-09-13 00:00:00,13.50,1,13.50,5,727,5
49577,1049578,1043935,10001,2012-09-12 00:00:00,2012-09-13 00:00:00,13.50,1,13.50,6,727,5


In [150]:
#df.ix[82969,]
df.head(5)


,orderlineid,orderid,productid,shipdate,billdate,unitprice,numunits,totalprice,No_cumm,total_trx,total_5trx
0,1010561,1006414,10834,2011-03-07 00:00:00,2011-03-08 00:00:00,18.00,1,18.00,1,3866,NaN
1,1010562,1006541,11052,2011-01-19 00:00:00,2011-01-20 00:00:00,10.00,2,20.00,2,2765,NaN
2,1010563,1006542,11070,2011-01-19 00:00:00,2011-01-20 00:00:00,19.12,1,19.12,1,2515,NaN
3,1010564,1010154,11196,2009-11-19 00:00:00,2009-11-20 00:00:00,14.95,1,14.95,1,9490,NaN
4,1010565,1009110,11048,2009-11-11 00:00:00,2009-11-12 00:00:00,10.00,2,20.00,2,4623,NaN


In [188]:
df = df.drop(['exclue_5_trx','total_5trx'],axis=1)

In [191]:
print(df[df['productid']==10001].head(6))

       orderlineid  orderid  productid             shipdate  \
48006      1048007  1042778      10001  2012-08-29 00:00:00   
48069      1048070  1042792      10001  2012-08-29 00:00:00   
48808      1048809  1042112      10001  2012-09-05 00:00:00   
49565      1049566  1042297      10001  2012-09-10 00:00:00   
49567      1049568  1043014      10001  2012-09-12 00:00:00   
49577      1049578  1043935      10001  2012-09-12 00:00:00   

                  billdate  unitprice  numunits  totalprice  No_cumm  \
48006  2012-08-30 00:00:00      13.35         1       13.35        1   
48069  2012-08-30 00:00:00      13.50         1       13.50        2   
48808  2012-09-06 00:00:00      13.35         1       13.35        3   
49565  2012-09-11 00:00:00      13.50         1       13.50        4   
49567  2012-09-13 00:00:00      13.50         1       13.50        5   
49577  2012-09-13 00:00:00      13.50         1       13.50        6   

       total_trx  total_5trx  total_5amt  
48006     

In [190]:
df.reset_index()

df5 = df.sort_values(by=['productid']).groupby(['productid']).head(5).groupby(['productid']).agg({'numunits': np.sum,'totalprice':np.sum}).reset_index()

df5 = df5.rename(columns={'numunits':'total_5trx','totalprice':'total_5amt'})

df = pd.merge(df,df5,on='productid',how='left') 
print(df5.head(10))


   productid  total_5trx  total_5amt
0      10001           5       68.55
1      10002           8       87.60
2      10003          11      117.15
3      10004           5      380.18
4      10005           5       76.80
5      10006           6       44.25
6      10007           5       38.75
7      10008           5       54.75
8      10009           9       92.55
9      10010           5       57.75


In [19]:
df['exclue_5_trx'] = df['total_trx'] - df['total_5trx']


KeyError: 'total_5trx'

In [ ]:
df['exclude_5_amt'] = df[df['total_5trx']>df[]]

In [264]:
df10006 = df[df['productid']==10006].sort_values(['rankProd'],ascending=1)

print(df10006[['productid','billdate','numunits','totalprice','No_cumm','total_trx','rankProd']][50:60])

       productid             billdate  numunits  totalprice  No_cumm  \
40262      10006  2010-11-25 00:00:00         1         7.5       51   
40349      10006  2010-11-25 00:00:00         1         7.5       52   
40418      10006  2010-11-25 00:00:00         1         7.5       53   
40464      10006  2010-11-25 00:00:00         1         7.5       54   
40467      10006  2010-11-25 00:00:00         1         7.5       55   
40558      10006  2010-11-25 00:00:00         1         7.5       56   
40559      10006  2010-11-25 00:00:00         1         7.5       57   
40567      10006  2010-11-25 00:00:00         1         7.5       58   
40650      10006  2010-11-25 00:00:00         1         7.5       59   
40652      10006  2010-11-25 00:00:00         1         7.5       60   

       total_trx  rankProd  
40262       2329        51  
40349       2329        52  
40418       2329        53  
40464       2329        54  
40467       2329        55  
40558       2329        56  
4055

In [259]:
df10006 = df[df['productid']==10004].sort_values(['rankProd'],ascending=1).head(50)

print(df10006[['productid','billdate','numunits','totalprice','No_cumm','total_trx','rankProd']].head(50))

        productid             billdate  numunits  totalprice  No_cumm  \
123587      10004  2013-08-27 00:00:00         1       90.95        1   
123960      10004  2013-08-27 00:00:00         1       77.31        2   
124005      10004  2013-08-27 00:00:00         1       72.76        3   
124007      10004  2013-08-27 00:00:00         1       72.76        4   
124094      10004  2013-08-27 00:00:00         1       72.75        5   
124097      10004  2013-08-27 00:00:00         1       72.76        6   
124315      10004  2013-08-27 00:00:00         1       72.76        7   
124316      10004  2013-08-27 00:00:00         1       72.76        8   
124317      10004  2013-08-27 00:00:00         1       72.76        9   
124349      10004  2013-08-27 00:00:00         1       72.76       10   
124365      10004  2013-08-27 00:00:00         1       72.76       11   
124367      10004  2013-08-27 00:00:00         1       72.76       12   
124413      10004  2013-08-27 00:00:00         1   

In [212]:
df.groupby('productid').agg({'rankProd':[np.min,np.max]}).reset_index().head(10)

productid rankProd      
                amin  amax
0     10001        1   256
1     10002        1   230
2     10003        1   172
3     10004        1    34
4     10005        1  1703
5     10006        1  1524
6     10007        1  2546
7     10008        1  1290
8     10009        1    49
9     10010        1   668

In [271]:
df['gt_5_nu'] =  df[df['rankProd']<=5].groupby(['productid'])['numunits'].transform(np.sum)

In [279]:
print(df[df['productid']==10006].sort_values(['rankProd'],ascending=1).head(10))

      orderlineid  orderid  productid             shipdate  \
1368      1000049  1000066      10006  2010-11-24 00:00:00   
1379      1000060  1000037      10006  2011-01-11 00:00:00   
1383      1000064  1000104      10006  2011-01-11 00:00:00   
1408      1000089  1000124      10006  2010-11-24 00:00:00   
1421      1000102  1000108      10006  2011-01-11 00:00:00   
1435      1000116  1000138      10006  2010-11-24 00:00:00   
1462      1000143  1000114      10006  2010-12-23 00:00:00   
1469      1000150  1000333      10006  2010-11-24 00:00:00   
1517      1000198  1000367      10006  2010-11-24 00:00:00   
1518      1000199  1000257      10006  2010-12-17 00:00:00   

                 billdate  unitprice  numunits  totalprice  No_cumm  \
1368  2010-11-25 00:00:00       7.50         1        7.50        1   
1379  2011-01-12 00:00:00       6.75         1        6.75      871   
1383  2011-01-12 00:00:00       7.50         1        7.50      872   
1408  2010-11-25 00:00:00       7

In [283]:
dftmp = df[df['rankProd']<=8].groupby(['productid']).agg({'numunits':np.sum}).reset_index()

In [284]:
dftmp.head(10)

,productid,numunits
0,10001,8
1,10002,27
2,10003,13
3,10004,8
4,10005,8
5,10006,10
6,10007,8
7,10008,13
8,10009,8
9,10010,8
